# Interactive Analytics
<br>
## Task
Run interactive queries on the data and answer some analytical questions

## Data
* use two datasets about influencers
* the first dataset contains basic information about each influencer
* the second dataset contains posting history for each influncer for the past 6 months

## Questions:
1) Count number of distinct languages used in the dataset

2) List all interests in descending order depending on its frequency

3) How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june

4) For each influencer add a new column n_posts with number of posts he published in its history 

5) Find top 10 influencers that posted the most posts in their history

6) Compute entropy of the n_posts column for each interest

## Documentation
<br>
* Pyspark documentation of DataFrame API is <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html">here</a>

* Prezentation slides are accessed <a target="_blank" href = "https://docs.google.com/presentation/d/1XNKIfE5Atj_Mzse0wjmbwLecmVs2YkWm9cqOLqDVWPo/edit?usp=sharing">here</a>

Difficulty measure of questions:
* one star means very basic
* five stars means difficult

### Import functions and modules

In [5]:
from pyspark.sql.functions import count, col, size, greatest, collect_list, when, explode, sum, rank, dense_rank, desc, row_number, avg, pandas_udf, PandasUDFType, array_contains, array_sort, lit, dayofweek
from pyspark.sql import Window
from pyspark.sql.types import DateType, DoubleType, StructType, StructField, DecimalType, LongType, StringType

import scipy.stats

### Load the data

In [7]:
infls = spark.table('mlprague.influencers')

posts_history = spark.table('mlprague.infl_posting_history')

### Explore the data

hint
* check the schema call df.printSchema()
* check the row count - call df.count()
* see some rows - call df.show(), or display(df)

In [9]:
# your code here:
infls.printSchema()
infls.count()
infls.show()
display(infls)

influencer_id,languages,interests
85194,List(en),"List(Family and relationships, Entertainment)"
67648,List(fr),List(Shopping and fashion)
184917,List(en),"List(Hobbies and activities, Food and drink, Family and relationships)"
370126,List(en),List(Hobbies and activities)
246388,List(es),"List(Sports and outdoors, Fitness and wellness)"
228457,List(en),"List(Entertainment, Hobbies and activities)"
122770,List(id),"List(Business and industry, Hobbies and activities)"
41916,List(ru),"List(Food and drink, Business and industry, Family and relationships)"
269152,List(hi),"List(Family and relationships, Hobbies and activities)"
204386,List(ru),List(Hobbies and activities)


### Count number of distinct languages used in the dataset (*)

hint
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode">explode</a> and <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct">distinct</a>

In [11]:
# your code here:
infls.select(explode("languages")).distinct().count()

Out[ 32 ]: 91

### List all interests in descending order depending on its frequency (\* \*)

hint:
* use explode on the array
* use groupBy with some aggregation function (slide 25 in the presentation might be useful for aggregations)
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [13]:
# your code here:
infls.select(explode("interests")).groupBy("col").count().orderBy("count",ascending=False).show()

+--------------------+------+
 col| count|
+--------------------+------+
Hobbies and activ...|197725|
Shopping and fashion|122291|
Family and relati...|106460|
 Entertainment| 80037|
Business and indu...| 79194|
 Food and drink| 50671|
Fitness and wellness| 39184|
 Sports and outdoors| 28621|
 Technology| 10751|
+--------------------+------+

### How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june (\* \* \*)

hint
* Possible way how to do it:
 * join influencers with their posting history
 * create 2 dataframes - in the first one select influencers that published in july, in the second select influencers that published in june
 * apply filters for interest and language
 * do a left anti join ('left_anti')
 * deduplicate the result
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">joins</a> in the docs
* slide 20 in the presentation might be useful for joins
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates">dropDuplicates</a> in the docs

In [15]:
# your code here:


### For each influencer add a new column n_posts with number of posts he published in its history (\* \*)

hint
* This question is prerequisite for the two next questions
* Possible way how to do it:
 * aggregate the posting history and join the result to influencers

In [17]:
# your code here:

### Find top 10 influencers that posted the most posts in their history (*)

hint
* this question depends on the result from the previous question
* orderBy the previous result

In [19]:
# your code here:

### Compute entropy of the n_posts column for each interest (\* \* \* \* \*)

hint
* This question depends on results from the previous 2 questions
* There are at least 2 ways how you can solve this (choose which way you prefer)
* Way 1:
 * Using higer order functions (slide 55 in the presentation might be useful)
 * first compute probability for each interest (for each interest get an array with probabilities of n_posts col)
 * (you might want to groupBy interest and n_posts than use a window and finaly groupBy again by interest and use collect_list for final aggregation)
 * use AGGREGATE in SQL expression to compute the entropy from the array
* Way 2:
  * Using Pandas grouped_map UDF (slide 40 in the presentation might be useful)
  * Define schema for the udf
  * Implement the udf (to compute the probabilities you can use value_counts() of Pandas.Series and divide by number of rows using len(pandas dataframe))
  * Use scipy.stats.entropy for the entropy calculation
  
Note
* For entropy calculation use the same definition as is used in scipy: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html

$$S = - \Sigma_{i} \left(p\left(i\right) * \log\left(p\left(i\right)\right)\right)$$

* for probability computation you can use this approach: compute the frequency of each value and divide by the total number of values
 * example: suppose we have this array [10, 10, 11, 12], the probability of 10 is p(10) = 2/4 = 0.5, the probability of 11 is p(11) = 1/4 = 0.25

In [21]:
# Using Higher order functions:

# your code here:

In [22]:
# using Pandas UDF:

# your code here:

### You may want to take a look at the distribution for given interests

In [24]:
# your code here: